In [1]:
# === SEKCJA 1: IMPORT I WCZYTANIE DANYCH ===
import pandas as pd
import numpy as np
import re
from IPython.display import display
import pickle
import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

print("--- Wczytywanie danych ---")
try:
    df_main_raw = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    # Wczytujemy nasz nowy, zweryfikowany słownik
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print("Pliki wczytane pomyślnie.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku: {e.filename}.")
    raise

C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


--- Wczytywanie danych ---
Pliki wczytane pomyślnie.


In [2]:
# === SEKCJA 2: PRZYGOTOWANIE DANYCH -- Z WZBOGACANIEM OPISU I NOWYMI CECHAMI ===
print("--- Przygotowanie danych do modelu ---\n")

# --- 2.1: Przygotowanie danych z ofert ---
df_main = df_main_raw.copy()
df_main.columns = [i for i in range(53)] + ['WojewodztwoID', 'PowiatID', 'GminaID', 'RodzajGminyID', 'MiastoID', 'DzielnicaID', 'UlicaID']

# === ZMIANA: Dodano mapowanie dla 'Title' i 'BuiltYear' ===
# !!! WAŻNE: Sprawdź, czy indeksy (3 i 39) są poprawne dla Twojego pliku CSV !!!
main_cols_map = {
    0: 'SaleId',
    3: 'Title',
    4: 'Description',
    5: 'Area',
    6: 'Price',
    12: 'BuiltYear',     # <-- POPRAWNIE
    17: 'NumberOfRooms',
    35: 'Floor',
    36: 'Floors'
}
df_main.rename(columns=main_cols_map, inplace=True)

# === ZMIANA: Dodano 'BuiltYear' do cech numerycznych ===
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
text_features = ['Title', 'Description'] # <-- DODANO dla przejrzystości
id_features = ['UlicaID']

for col in numeric_features + id_features:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')

# Upewnijmy się, że kluczowe kolumny tekstowe nie są puste i zastąpmy ewentualne NaN
for col in text_features:
    df_main[col] = df_main[col].fillna('')

# Usuwamy wiersze, gdzie brakuje kluczowych danych numerycznych lub ID
df_main.dropna(subset=numeric_features + id_features, inplace=True)
df_main['UlicaID'] = df_main['UlicaID'].astype(int)

# --- 2.2: Łączenie ofert ze słownikiem ---
print("\nŁączenie ofert z danymi ze słownika...")
df_merged = pd.merge(df_main, df_slownik, on='UlicaID', how='inner')
print(f"Liczba ofert po połączeniu ze słownikiem: {len(df_merged)}")

if len(df_merged) == 0:
    raise ValueError("Połączenie danych nie dało żadnych wyników.")

df_model_ready = df_merged.copy()
print(f"Finalny zbiór danych gotowy. Wiersze: {len(df_model_ready)}")


# --- 2.3: Przygotowanie Danych Wejściowych (X) ---
# ==============================================================================
# === ZMIANA: Wzbogacanie opisu o TYTUŁ i nazwy lokalizacji ===
# ==============================================================================
print("\nWzbogacanie opisów o tytuł i nazwy lokalizacji w celu wzmocnienia sygnału...")
df_model_ready['description_enriched'] = df_model_ready['Title'] + " " + df_model_ready['Description'] + " " + df_model_ready['Dzielnica_Name'] + " " + df_model_ready['Ulica_Name']

def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())

# Używamy teraz nowej, wzbogaconej kolumny do nauki tokenizera
df_model_ready['description_clean'] = df_model_ready['description_enriched'].apply(clean_text)

# ==============================================================================

MAX_WORDS, MAX_LEN = 20000, 250
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_model_ready['description_clean'])
X_text = pad_sequences(tokenizer.texts_to_sequences(df_model_ready['description_clean']), maxlen=MAX_LEN)

df_model_ready['Price_per_sqm'] = df_model_ready['Price'] / df_model_ready['Area']
df_model_ready['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)

# === ZMIANA: Dodano 'BuiltYear' do listy cech dla pipelinu numerycznego ===
numeric_features_cols = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear', 'Price_per_sqm']
numeric_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_numeric = numeric_pipeline.fit_transform(df_model_ready[numeric_features_cols])

# --- 2.4: Przygotowanie Danych Wyjściowych (y) ---
le_dzielnica = LabelEncoder()
y_dzielnica = le_dzielnica.fit_transform(df_model_ready['Dzielnica_Name'])
num_classes_dzielnica = len(le_dzielnica.classes_)
le_ulica = LabelEncoder()
y_ulica = le_ulica.fit_transform(df_model_ready['Ulica_Name'])
num_classes_ulica = len(le_ulica.classes_)

print(f"\nProblem przygotowany do modelowania:")
print(f" - Liczba klas (dzielnice): {num_classes_dzielnica} -> {le_dzielnica.classes_[:5]}...")
print(f" - Liczba klas (ulice): {num_classes_ulica}")

train_indices, val_indices = train_test_split(range(len(df_model_ready)), test_size=0.2, random_state=42, stratify=y_dzielnica)
X_train_text, X_val_text = X_text[train_indices], X_text[val_indices]
X_train_num, X_val_num = X_numeric[train_indices], X_numeric[val_indices]
y_train_dzielnica, y_val_dzielnica = y_dzielnica[train_indices], y_dzielnica[val_indices]
y_train_ulica, y_val_ulica = y_ulica[train_indices], y_ulica[val_indices]

print("\nDane podzielone na zbiory treningowe i walidacyjne.")

--- Przygotowanie danych do modelu ---


Łączenie ofert z danymi ze słownika...
Liczba ofert po połączeniu ze słownikiem: 8066
Finalny zbiór danych gotowy. Wiersze: 8066

Wzbogacanie opisów o tytuł i nazwy lokalizacji w celu wzmocnienia sygnału...

Problem przygotowany do modelowania:
 - Liczba klas (dzielnice): 18 -> ['Bemowo' 'Białołęka' 'Bielany' 'Mokotów' 'Ochota']...
 - Liczba klas (ulice): 661

Dane podzielone na zbiory treningowe i walidacyjne.


In [3]:
# === SEKCJA 3: BUDOWA I TRENING MODELU HIERARCHICZNEGO ===

# --- 3.1: Definicja architektury ---
# Wejścia
input_text = Input(shape=(MAX_LEN,), name='text_input')
input_numeric = Input(shape=(X_numeric.shape[1],), name='numeric_input')

# Wspólny trzon
text_embedding = Embedding(input_dim=MAX_WORDS, output_dim=128)(input_text)
lstm_out = LSTM(128, dropout=0.3)(text_embedding)
concatenated = Concatenate()([lstm_out, input_numeric])
common_dense = Dense(128, activation='relu')(concatenated)
common_dense = Dropout(0.5)(common_dense)

# Gałąź wyjściowa dla DZIELNICY
dzielnica_branch = Dense(64, activation='relu')(common_dense)
dzielnica_output = Dense(num_classes_dzielnica, activation='softmax', name='output_dzielnica')(dzielnica_branch)

# Gałąź wyjściowa dla ULICY
ulica_branch = Dense(256, activation='relu')(common_dense)
ulica_output = Dense(num_classes_ulica, activation='softmax', name='output_ulica')(ulica_branch)

# --- 3.2: Kompilacja modelu ---
model = Model(inputs=[input_text, input_numeric], outputs=[dzielnica_output, ulica_output])

# Definiujemy osobne straty dla każdego wyjścia
losses = {
    "output_dzielnica": "sparse_categorical_crossentropy",
    "output_ulica": "sparse_categorical_crossentropy",
}

# Definiujemy wagi dla każdej ze strat
loss_weights = {
    "output_dzielnica": 1.0,
    "output_ulica": 0.5
}

# POPRAWKA: Definiujemy metryki dla każdego wyjścia osobno
metrics = {
    "output_dzielnica": "accuracy",
    "output_ulica": "accuracy"
}

model.compile(
    optimizer='adam',
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics  # Przekazujemy słownik metryk
)
model.summary()

# --- 3.3: Trening ---
X_train = [X_train_text, X_train_num]
y_train = {'output_dzielnica': y_train_dzielnica, 'output_ulica': y_train_ulica}
X_val = [X_val_text, X_val_num]
y_val = {'output_dzielnica': y_val_dzielnica, 'output_ulica': y_val_ulica}

callbacks = [
    EarlyStopping(
        monitor='val_output_dzielnica_accuracy', 
        patience=5, 
        restore_best_weights=True, 
        verbose=1,
        mode='max'  # <-- DODAJ TĘ LINIĘ
    ),
    ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
]

print("\nRozpoczynam trening modelu hierarchicznego...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 250)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 250, 128)          │       2,560,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 128)               │         131,584 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ numeric_input (InputLayer)    │ (None, 7)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 135)               │               0 │ lstm[0][0],                │
│                               │                           │                 │ numeric_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │          17,408 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 256)               │          33,024 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dzielnica (Dense)      │ (None, 18)                │           1,170 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_ulica (Dense)          │ (None, 661)               │         169,877 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,921,319 (11.14 MB)

 Trainable params: 2,921,319 (11.14 MB)

 Non-trainable params: 0 (0.00 B)


Rozpoczynam trening modelu hierarchicznego...
Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 23s 393ms/step - loss: 5.4596 - output_dzielnica_accuracy: 0.2667 - output_dzielnica_loss: 2.4988 - output_ulica_accuracy: 0.0816 - output_ulica_loss: 5.9209 - val_loss: 3.5376 - val_output_dzielnica_accuracy: 0.5892 - val_output_dzielnica_loss: 1.2953 - val_output_ulica_accuracy: 0.1648 - val_output_ulica_loss: 4.4905 - learning_rate: 0.0010
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 19s 369ms/step - loss: 3.2336 - output_dzielnica_accuracy: 0.6756 - output_dzielnica_loss: 1.0819 - output_ulica_accuracy: 0.1759 - output_ulica_loss: 4.3029 - val_loss: 2.4042 - val_output_dzielnica_accuracy: 0.8309 - val_output_dzielnica_loss: 0.5542 - val_output_ulica_accuracy: 0.1958 - val_output_ulica_loss: 3.7026 - learning_rate: 0.0010
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 19s 366ms/step - loss: 2.2555 - output_dzielnica_accuracy: 0.8426 - output_dzielnica_loss: 0.5010 - output_ulica_accuracy: 0.2445 - output_ulica_loss

In [4]:
# === SEKCJA 4: ZAPIS ARTEFAKTÓW DO PRODUKCJI ===
artifacts_dir = 'model_artifacts_final'
os.makedirs(artifacts_dir, exist_ok=True)

# 1. Zapis modelu
model.save(os.path.join(artifacts_dir, 'final_hierarchical_model.keras'))

# 2. Zapis Tokenizera
with open(os.path.join(artifacts_dir, 'tokenizer.pkl'), 'wb') as f: pickle.dump(tokenizer, f)

# 3. Zapis pipelinu numerycznego
with open(os.path.join(artifacts_dir, 'numeric_pipeline.pkl'), 'wb') as f: pickle.dump(numeric_pipeline, f)

# 4. Zapis koderów dla zmiennych celu
with open(os.path.join(artifacts_dir, 'le_dzielnica.pkl'), 'wb') as f: pickle.dump(le_dzielnica, f)
with open(os.path.join(artifacts_dir, 'le_ulica.pkl'), 'wb') as f: pickle.dump(le_ulica, f)

print(f"\nWszystkie artefakty zostały zapisane w folderze: '{artifacts_dir}'")


Wszystkie artefakty zostały zapisane w folderze: 'model_artifacts_final'


In [7]:
# === SEKCJA 5: PREdykcja na pełnym zbiorze danych i interpretacja wyników ===

import pandas as pd
import numpy as np
import pickle
import os
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython.display import display

print("--- Rozpoczynam proces predykcji na pełnym zbiorze danych ---")

# --- 5.1: Wczytanie zapisanych artefaktów ---
artifacts_dir = 'model_artifacts_final'
try:
    print(f"Wczytuję artefakty z folderu: '{artifacts_dir}'...")
    model = load_model(os.path.join(artifacts_dir, 'final_hierarchical_model.keras'))
    with open(os.path.join(artifacts_dir, 'tokenizer.pkl'), 'rb') as f:
        tokenizer = pickle.load(f)
    with open(os.path.join(artifacts_dir, 'numeric_pipeline.pkl'), 'rb') as f:
        numeric_pipeline = pickle.load(f)
    with open(os.path.join(artifacts_dir, 'le_dzielnica.pkl'), 'rb') as f:
        le_dzielnica = pickle.load(f)
    with open(os.path.join(artifacts_dir, 'le_ulica.pkl'), 'rb') as f:
        le_ulica = pickle.load(f)
    print("Artefakty wczytane pomyślnie.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku z artefaktem: {e.filename}. Upewnij się, że model został poprawnie wytrenowany i zapisany.")
    raise

# --- 5.2: Wczytanie i przygotowanie pełnego zbioru danych ---
# Używamy df_main_raw, który powinien być w pamięci z komórki #1, aby nie wczytywać pliku ponownie.
print(f"\nPrzygotowuję {len(df_main_raw)} wierszy do predykcji...")
df_to_predict = df_main_raw.copy()

# KROK 1: Użyj IDENTYCZNEGO mapowania kolumn jak w treningu
main_cols_map = {
    0: 'SaleId',
    3: 'Title',
    4: 'Description',
    5: 'Area',
    6: 'Price',
    12: 'BuiltYear',     # <-- POPRAWIONY INDEKS
    17: 'NumberOfRooms',
    35: 'Floor',
    36: 'Floors'
}
df_to_predict.rename(columns=main_cols_map, inplace=True)

# KROK 2: Wykonaj IDENTYCZNE przetwarzanie cech jak w treningu
text_features = ['Title', 'Description']
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']

for col in text_features:
    df_to_predict[col] = df_to_predict[col].fillna('')
for col in numeric_features:
    df_to_predict[col] = pd.to_numeric(df_to_predict[col], errors='coerce')

# KROK 3: Odfiltruj wiersze, które nie mają kluczowych danych numerycznych
df_valid_for_pred = df_to_predict.dropna(subset=numeric_features).copy()
print(f"Znaleziono {len(df_valid_for_pred)} wierszy, na których można wykonać predykcję.")

# KROK 4: Stwórz dodatkowe cechy, tak jak w treningu
# A. Stwórz cechę 'Price_per_sqm'
df_valid_for_pred['Price_per_sqm'] = df_valid_for_pred['Price'] / df_valid_for_pred['Area']
df_valid_for_pred['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)

# B. Wzbogać i wyczyść opis. UWAGA: Tutaj nie dodajemy Dzielnicy i Ulicy, bo ich nie znamy!
df_valid_for_pred['description_enriched'] = df_valid_for_pred['Title'] + " " + df_valid_for_pred['Description']
def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
df_valid_for_pred['description_clean'] = df_valid_for_pred['description_enriched'].apply(clean_text)

# --- 5.3: Transformacja danych przy użyciu wczytanych artefaktów ---
MAX_LEN = 250 # Musi być takie samo jak podczas treningu

# A. Przetwarzanie danych tekstowych
X_text_pred = pad_sequences(tokenizer.texts_to_sequences(df_valid_for_pred['description_clean']), maxlen=MAX_LEN)

# B. Przetwarzanie danych numerycznych
# Lista kolumn MUSI być identyczna jak w treningu
numeric_features_cols_pipeline = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear', 'Price_per_sqm']
X_numeric_pred = numeric_pipeline.transform(df_valid_for_pred[numeric_features_cols_pipeline])

# --- 5.4: Wykonanie predykcji ---
print("\nRozpoczynam predykcję modelem LSTM...")
predictions = model.predict([X_text_pred, X_numeric_pred])
pred_dzielnica_probs = predictions[0]
pred_ulica_probs = predictions[1]
print("Predykcja zakończona.")

# === SEKCJA 5.5: INTELIGENTNA INTERPRETACJA WYNIKÓW (z korektą) ===

# Krok 1: Przewidujemy dzielnicę (tak jak wcześniej)
pred_dzielnica_indices = np.argmax(pred_dzielnica_probs, axis=1)
predicted_dzielnica_names = le_dzielnica.inverse_transform(pred_dzielnica_indices)
df_valid_for_pred['predicted_dzielnica'] = predicted_dzielnica_names

# Krok 2: Przygotowujemy dane do inteligentnej predykcji ulicy
# Mapowanie: Nazwa dzielnicy -> lista poprawnych nazw ulic
dzielnica_to_ulice_map = df_slownik.groupby('Dzielnica_Name')['Ulica_Name'].apply(list).to_dict()
# Mapowanie: Nazwa ulicy -> jej indeks w modelu
ulica_name_to_idx_map = {name: i for i, name in enumerate(le_ulica.classes_)}

corrected_ulica_indices = []
# Iterujemy przez każdą predykcję
for i in range(len(predicted_dzielnica_names)):
    # Bierzemy przewidzianą dzielnicę dla i-tego wiersza
    dzielnica = predicted_dzielnica_names[i]
    
    # Pobieramy listę poprawnych ulic dla tej dzielnicy
    valid_ulica_names = dzielnica_to_ulice_map.get(dzielnica, [])
    
    # Konwertujemy nazwy ulic na ich indeksy, których używa model
    valid_ulica_indices = [ulica_name_to_idx_map[name] for name in valid_ulica_names if name in ulica_name_to_idx_map]
    
    if not valid_ulica_indices:
        # Jeśli z jakiegoś powodu nie ma ulic dla dzielnicy, wybierz po prostu najlepszą globalnie
        corrected_ulica_indices.append(np.argmax(pred_ulica_probs[i]))
        continue

    # Bierzemy pełny wektor prawdopodobieństw dla ulic dla i-tego wiersza
    all_ulica_probs = pred_ulica_probs[i]
    
    # Wybieramy prawdopodobieństwa tylko dla poprawnych ulic
    valid_probs = all_ulica_probs[valid_ulica_indices]
    
    # Znajdujemy indeks NAJLEPSZEJ ulicy WŚRÓD poprawnych ulic
    best_local_idx = np.argmax(valid_probs)
    
    # Tłumaczymy ten lokalny indeks z powrotem na globalny indeks ulicy
    final_ulica_idx = valid_ulica_indices[best_local_idx]
    
    corrected_ulica_indices.append(final_ulica_idx)

# Krok 3: Użyj skorygowanych indeksów do finalnej predykcji
df_valid_for_pred['predicted_ulica'] = le_ulica.inverse_transform(corrected_ulica_indices)

# --- 5.6: Wyświetlenie wyników ---
print("\nPrzykładowe wyniki predykcji:")
display(df_valid_for_pred[[
    'Title',
    'Area',
    'Price',
    'BuiltYear',
    'predicted_dzielnica',
    'predicted_ulica'
]].head(20))

--- Rozpoczynam proces predykcji na pełnym zbiorze danych ---
Wczytuję artefakty z folderu: 'model_artifacts_final'...
Artefakty wczytane pomyślnie.

Przygotowuję 235700 wierszy do predykcji...
Znaleziono 179492 wierszy, na których można wykonać predykcję.

Rozpoczynam predykcję modelem LSTM...
5610/5610 ━━━━━━━━━━━━━━━━━━━━ 359s 64ms/step
Predykcja zakończona.

Przykładowe wyniki predykcji:


,Title,Area,Price,BuiltYear,predicted_dzielnica,predicted_ulica
0,"Mieszkanie, Mokotów",167.25,2893425.0,2022.0,Bielany,Kolektorska
1,Dwupoziomowy apartament w doskonałej lokalizacji,93.36,3500000.0,1966.0,Bielany,Kolektorska
2,Bezpośrednio! mieszkanie - Wilanów,41.00,649000.0,2017.0,Praga-południe,Drewnicka
4,"Mieszkanie 3-pokoje, umeblowane, po remoncie",62.00,880000.0,1960.0,Mokotów,Sozopolska
5,PREMIUM !!! Apartament z Widokiem 22Piętro!Taras!,46.00,850000.0,2021.0,Żoliborz,Kolektorska
6,"Mieszkanie Bródno, ul. Wysockiego",53.50,650000.0,1974.0,Żoliborz,Bieniewicka
7,Moja Północna II | mieszkanie B 3,41.05,496705.0,2023.0,Ursus,Quo vadis
12,"Mieszkanie Warszawa Śródmieście, ul. Jana Pawł...",64.00,879000.0,1953.0,Żoliborz,Gwiaździsta
14,3 pokoje Mokotów Woronicza/Suwak,75.00,1200000.0,2020.0,Żoliborz,Bieniewicka
15,Mieszkanie trzypokojowe na sprzedaż,100.00,799000.0,1996.0,Praga-południe,Podolska
